# Animation of the PSF of dancing interferometers

_M Kenworthy_ Leiden Observatory kenworthy@strw.leidenuniv.nl

Based on hcipy tutorials https://docs.hcipy.org/0.3.1/tutorials/index.html and examples, and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/


In [1]:
#pip install progressbar2

In [2]:
#pip install ffpmeg

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from hcipy import *
from scenes import *

from progressbar import progressbar 

anim_version='v1'

In [4]:
from matplotlib import animation, rc
#from IPython.display import HTML

In [ ]:

class Dish:
    def __init__(x=0.0, y=0.0, d=0.1, ampl=1.0):
        'Dish - a single element in a telescope array'
        self.x = x
        self.y = y
        self.d = d
        self.ampl = ampl
        
        set_polar(self, r, theta):
            'set position of Dish with polar coordinates'
            self.x = r*np.cos(theta)
            self.y = r*np.sin(theta)
            


In [ ]:
ndish = 5

ddish = .1

dish_x = np.linspace(-0.5+(ddish/2.), 0.5-(ddish/2.), ndish)

dish_y = np.zeros_like(dish_x)

interf_array = UnstructuredCoords((dish_x,dish_y))

grid = Grid(interf_array)

dish = circular_aperture(ddish) # this is a Field generator

interferom = make_segmented_aperture(dish, grid, np.array([1.0]))

pupil_grid = make_pupil_grid(1024,1.5)
inter_pupil = evaluate_supersampled(interferom, pupil_grid, 8)

imshow_field(inter_pupil)

In [ ]:
focal_grid = make_focal_grid(8, 14)

prop = FraunhoferPropagator(inter_pupil, focal_grid)

wf = Wavefront(inter_pupil)
focal_image = prop(wf).intensity
imshow_field(np.log10(focal_image / focal_image.max()), vmin=-5, cmap='inferno')

## HCIpy setup

In [ ]:
ddish = Stage()
ddish.add(Act(1.0, 0.05, 5,  'sig',10))
ddish.add(Act(0.05, 0.1, 5,  'sig',10))
ddish.add(Act(0.1, 0.2, 5,  'sig',10))

print(ddish.total_time())

t = np.arange(0,ddish.total_time(),0.05)

plt.plot(t, ddish.t(t))

In [ ]:
total_time_animation = ddish.total_time() # seconds
frame_rate = 10          # frames per second
total_frames = total_time_animation * frame_rate # seconds
supersampling = 4

plt.figure(figsize=(8,4))

anim = FFMpegWriter('interferometers_{}.mp4'.format(anim_version), framerate=frame_rate)

times = np.arange(0,total_time_animation, 1./frame_rate)

dd = ddish.t(times)

ndish = 5
pupil_grid = make_pupil_grid(1024,1.5)
focal_grid = make_focal_grid(8, 14)

for t in progressbar(range(np.int(total_frames))):

    dish = circular_aperture(dd[t]) # this is a Field generator

    dish_x = np.linspace(-0.5+(dd[t]/2.), 0.5-(dd[t]/2.), ndish)

    dish_y = np.zeros_like(dish_x)

    interf_array = UnstructuredCoords((dish_x,dish_y))

    grid = Grid(interf_array)
    
    interferom = make_segmented_aperture(dish, grid, np.array([1.0]))

    inter_pupil = evaluate_supersampled(interferom, pupil_grid, supersampling)

    plt.clf()
#    plt.suptitle('Timestep %d / %d' % (t, total_frames))
    
    plt.subplot(1,2,1)
    imshow_field(inter_pupil, cmap='gray')
    plt.title('Aperture')
    
    plt.text(0.05,0.05,anim_version, fontsize=12, color='white',
            horizontalalignment='left', verticalalignment='bottom',
            transform=plt.gca().transAxes)
    
    prop = FraunhoferPropagator(inter_pupil, focal_grid)

    wf = Wavefront(inter_pupil)
    focal_image = prop(wf).intensity
    
    plt.subplot(1,2,2)
    imshow_field(np.log10(focal_image / focal_image.max()), vmin=-5, cmap='inferno')
    #plt.title('frame {} at time {:.2f}'.format(timestep, timestep/frame_rate))
    plt.title('PSF')

    plt.text(0.95,0.05,"@mattkenworthy", fontsize=12, color='white',
        horizontalalignment='right', verticalalignment='bottom',
        transform=plt.gca().transAxes)
    
    anim.add_frame()

plt.close()
anim.close()

# Show created animation
anim

In [ ]:
!ffmpeg -i interferometers_v1.mp4 -vf "fps=10,scale=400:-1:flags=lanczos" -c:v pam -f image2pipe - | convert -delay 10 - -loop 0 -layers optimize interferometers_v1.gif